<a href="https://colab.research.google.com/github/caguirre5/Lab10-DL/blob/main/Lab10_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Universidad del Valle <br>
Deep Learning <br>
Integrantes:

* Marco Jurado 20308
* Diego Cordova 20212
* Paola Contreras 20213
* Paola de Leon 20361
* Cristian Aguirre 20231

# Parte 1. Practica

In [1]:
import gym
import numpy as np

# Parte 2. Teoria

Defina en qué consiste y en qué clase de problemas se pueden usar cada uno de los siguientes acercamientos en
Deep Reinforcement Learning


**1. Proximal Policy Optimization**



**2. Deep Deterministic Policy Gradients (DDPG)**



**3. Trust Region Policy Optimization (TRPO)**



**4. Asynchronous Advantage Actor-Critic (A3C)**

